In [41]:
import pandas as pd
import math
import os

In [42]:
# ---- Load the four main tables (TSV) ----
#incels_is_comments   = pd.read_csv("data/incels.is_AllComments.anon",           sep="\t", encoding="utf-8", on_bad_lines="skip")
reddit_incel   = pd.read_csv("data/reddit-incel-posts.anon.txt",          sep="\t", encoding="utf-8", dtype = str)
reddit_incelexit = pd.read_csv("data/reddit-IncelExit-posts.anon.txt",          sep="\t", encoding="utf-8", engine="python", quoting=3, on_bad_lines="skip", dtype=str)
#saidit_incel_posts   = pd.read_csv("data/saidit-incel-posts.anon.txt",          sep="\t", encoding="utf-8", on_bad_lines="skip")
reddit_braincels   = pd.read_csv("data/reddit-braincels-posts.anon.txt",          sep="\t", encoding="utf-8", on_bad_lines="skip", dtype=str)

reddit_braincels = reddit_braincels.iloc[1:].reset_index(drop=True)

# Optional: keep them in a dict for easy access
dfs = {
    #"incels_is_comments": incels_is_comments,
    "reddit_incel": reddit_incel,
    "reddit_incelexit": reddit_incelexit,
    #"saidit_incel_posts": saidit_incel_posts,
    "reddit_braincels": reddit_braincels
}

# streamline column names 
colnames = ["link", "comment_id", "user_id", "parent", "timestamp", "title", "text"]
for name, df in dfs.items():
    df.columns = colnames

# Quick peek so you can see they loaded
for name, df in dfs.items():
    print(f"{name}_posts: {df.shape[0]}")
    #display(df.head(10))

print("Unique users in each dataset:")
for name, df in dfs.items():
    print(f"{name}: {len(df['user_id'].unique())}")

#total unique users across all datasets
all_users = pd.concat([df['user_id'] for df in dfs.values()]).unique()
print(f"Total unique users across all datasets: {len(all_users)}")

#total length of all datasets combined 
total_length = sum(df.shape[0] for df in dfs.values())
print(f"Total length of all datasets combined: {total_length}")

reddit_incel_posts: 988061
reddit_incelexit_posts: 111083
reddit_braincels_posts: 2216973
Unique users in each dataset:
reddit_incel: 29877
reddit_incelexit: 4005
reddit_braincels: 69303
Total unique users across all datasets: 101200
Total length of all datasets combined: 3316117
Total unique users across all datasets: 101200
Total length of all datasets combined: 3316117


In [43]:
valid_posts_incelexit_posts_idx = reddit_incelexit['link'].str.contains(r'r/IncelExit')

valid_posts_incelexit = reddit_incelexit[valid_posts_incelexit_posts_idx]

valid_posts_incelexit['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', errors='coerce')
valid_posts_incelexit.sort_values('timestamp', inplace=True)

valid_posts_incelexit["Original File Source"] = "r/IncelExit"

print('Posts with valid link in IncelExit dataset:', valid_posts_incelexit_posts_idx.sum())
#valid_posts_incelexit.head() 

C:\Users\Renato Thieleke\AppData\Local\Temp\ipykernel_51364\1574897855.py:5: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  valid_posts_incelexit['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', errors='coerce')


Posts with valid link in IncelExit dataset: 110990


C:\Users\Renato Thieleke\AppData\Local\Temp\ipykernel_51364\1574897855.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_posts_incelexit['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', errors='coerce')
C:\Users\Renato Thieleke\AppData\Local\Temp\ipykernel_51364\1574897855.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_posts_incelexit.sort_values('timestamp', inplace=True)
C:\Users\Renato Thieleke\AppData\Local\Temp\ipykernel_51364\1574897855.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [44]:
input_path = "data/reddit-incel-posts.anon.txt"
output_path = "data/reddit-incel-posts_with_quotes.txt"

#adding quotation marks at the end of each
with open(input_path, encoding="utf-8", errors="ignore") as f_in, \
     open(output_path, "w", encoding="utf-8", newline="") as f_out:
    for line in f_in:
        if line.count('"') % 2 != 0:
            line = line.rstrip("\n") + '"\n'
        f_out.write(line)

valid_posts_incels = pd.read_csv(
    output_path,
    sep="\t",
    encoding="utf-8",
    dtype=str,
    header=None,
    names=colnames
)

valid_posts_incels['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', errors='coerce')
valid_posts_incels.sort_values('timestamp', inplace=True)

valid_posts_incels["Original File Source"] = "r/Incel"

print('Number of users r/Incel: ', len(valid_posts_incels['user_id'].unique()))
print("Number of posts r/Incel:", len(valid_posts_incels))

C:\Users\Renato Thieleke\AppData\Local\Temp\ipykernel_51364\2730981505.py:21: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  valid_posts_incels['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', errors='coerce')


Number of users r/Incel:  29877
Number of posts r/Incel: 988244


In [45]:
import csv
input_path = "data/reddit-braincels-posts.anon.txt"
output_path = "data/reddit-braincels-posts_with_quotes.txt"

#adding quotation marks at the end of each
with open(input_path, encoding="utf-8", errors="ignore") as f_in, \
     open(output_path, "w", encoding="utf-8", newline="") as f_out:
    for line in f_in:
        if line.count('"') % 2 != 0:
            line = line.rstrip("\n") + '"\n'
        f_out.write(line)

valid_posts_braincels = pd.read_csv(
    "data/reddit-braincels-posts_with_quotes.txt",
    sep="\t",
    encoding="utf-8",
    engine="python",
    quoting=csv.QUOTE_NONE,
    escapechar="\\",
    header=0,
    names=colnames,
    usecols=[0,1,2,3,4,5,6],
    dtype=str,
    na_filter=False,
    keep_default_na=False,
    on_bad_lines="warn", 
)

valid_posts_braincels['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', errors='coerce')
valid_posts_braincels.sort_values('timestamp', inplace=True)

valid_posts_braincels["Original File Source"] = "r/Braincels"

print('Number of users r/Braincel: ', len(valid_posts_braincels['user_id'].unique()))
print('Number of posts r/Braincel: ', len(valid_posts_braincels))

C:\Users\Renato Thieleke\AppData\Local\Temp\ipykernel_51364\1459181945.py:29: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  valid_posts_braincels['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', errors='coerce')


Number of users r/Braincel:  75267
Number of posts r/Braincel:  2408121


In [46]:
#Create compiled dataset
reddit_dataset_compiled = pd.concat([valid_posts_incels, valid_posts_incelexit, valid_posts_braincels])
print(reddit_dataset_compiled.shape)

#dropping rows for identified deleted user and posts
reddit_dataset_compiled = reddit_dataset_compiled[reddit_dataset_compiled['user_id'] != '924980486.069293']
reddit_dataset_compiled_clean = reddit_dataset_compiled[reddit_dataset_compiled['title'] != '924980486.069293'] 

print(reddit_dataset_compiled_clean.shape)

#Create column with the first subreddit each user posted in based on timestamp
first_posts = reddit_dataset_compiled_clean.sort_values('timestamp').groupby('user_id').first()['Original File Source'].reset_index()
first_posts.columns = ['user_id', 'Original User Source']

reddit_dataset_compiled_clean = reddit_dataset_compiled_clean.merge(first_posts, on='user_id', how='left')
print(reddit_dataset_compiled_clean.shape)

#Export to csv
reddit_dataset_compiled_clean.to_csv("data/reddit_dataset_compiled_clean.csv", sep=',', index=False, encoding='utf-8')
save_path = "data/reddit_dataset_compiled_clean.csv"

(3507355, 8)
(3098242, 8)
(3098242, 8)
(3098242, 9)
(3098242, 9)
